# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

Начинаем с добавления всех необходимых библиотек и функций.

In [1]:
# добавление библиотек и функций:
import pandas as pd
import numpy as np

import re

import nltk

from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier

from sklearn.metrics import f1_score

from sklearn.utils import shuffle

import warnings
warnings.filterwarnings('ignore')

Загрузим данные и посмотрим общую информацию.

In [2]:
# Загрузка данных:
df = pd.read_csv('/datasets/toxic_comments.csv')

display(df.head())
df.info()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


Сразу видим, что пропусков в данных нет. Проверим на дубликаты.

In [3]:
# Проверка на дубликаты:
df.duplicated().sum()

0

Кстати, интересно, какая часть комментариев помечена как токсичная.

In [4]:
# Небольшое исследование - сколько токсичных комментариев:
print('Токсичных комментариев от общего числа:', df['toxic'][df['toxic']==1].count()/df['toxic'].count())

Токсичных комментариев от общего числа: 0.10161213369158527


#### Промежуточный вывод:
Были загружены и осмотрены предоставленные данные. Пропусков и нулевых значений не обнаружено. Из интересного - доля токсичных комментариев от общего числа составила 10%; это может усложнить разбиение на выборки и дальнейшее обучение модели.

Приступим ко второй части предобработки. Сперва удалим лишний столбец Unnamed, который дублирует индексы.

In [5]:
# Удаление лишнего столбца:
df = df.drop('Unnamed: 0', axis=1)
display(df.head())

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


Теперь "очистим" данные от лишних символов и лемматизируем их. Для этого используем изученные функции работы с текстами.

In [6]:
# Обработка с помощью spacy:
import spacy
nlp = spacy.load("en_core_web_sm")

def clean_text(text):
    clean = nlp(" ".join(re.sub(r'[^a-zA-z]', ' ', text).split()))
    lemm_out = ' '.join([w.lemma_ for w in clean])
    return lemm_out

df['text'] = df['text'].apply(clean_text)
display(df.sample())

,text,toxic
89012,support split I d try condense it but it s rat...,0


In [7]:
sentence1 = "The striped bats are hanging on their feet for best"
sentence2 = "you should be ashamed of yourself went worked"
df_my = pd.DataFrame([sentence1, sentence2], columns = ['text'])
print(df_my)


print(df_my['text'].apply(clean_text))

                                                text
0  The striped bats are hanging on their feet for...
1      you should be ashamed of yourself went worked
0    the stripe bat be hang on their foot for good
1        you should be ashamed of yourself go work
Name: text, dtype: object


Осталось разделить данные на выборки и проследить, чтобы соотношение токсичных и обычных комментариев в тренировочной выборке было одинаковым.

In [8]:
# Разделение на выборки:
features = df.drop(['toxic'], axis=1) 
target = df.toxic
RANDOME_STATE = 12345

features_train, features_valid, target_train, target_valid = train_test_split(features, 
                                                                              target, 
                                                                              test_size=.2, 
                                                                              random_state=RANDOME_STATE)

features_valid, features_test, target_valid, target_test = train_test_split(features_valid, 
                                                                            target_valid, 
                                                                            test_size=.5,
                                                                            random_state=RANDOME_STATE)

In [9]:
# Смотрим соотношение в тренировочной выборке:
count_0 = len(target_train[target_train==0])
count_1 = len(target_train[target_train==1])

print('Доля значений 1 в тренировочной выборке:', count_1 / (count_1 + count_0))

Доля значений 1 в тренировочной выборке: 0.10166911239631807


Отношение такое же, как и в оригинальной выборке. Для адекватного обучения моделей необходимо сравнять его, механически перемешаем данные.

In [10]:
# Уменьшение "нулевых" строк:

df_train = df.iloc[target_train.index]
target_train_0 = df_train[df_train['toxic'] == 0]['toxic']
target_train_1 = df_train[df_train['toxic'] == 1]['toxic']


target_train_0_resample = target_train_0.sample(target_train_1.shape[0], random_state=RANDOME_STATE)
target_train_resample = pd.concat([target_train_0_resample, target_train_1])

features_train_resample = df.iloc[target_train_resample.index]

features_train_resample, target_train_resample = shuffle(features_train_resample,
                                                         target_train_resample,
                                                         random_state=RANDOME_STATE)

features_train_resample = features_train_resample.text 

display('Соотношение в тренировочной выборке:', target_train_resample.value_counts(normalize=True))

'Соотношение в тренировочной выборке:'

0    0.5
1    0.5
Name: toxic, dtype: float64

### Вывод:
Были загружены и осмотрены предоставленные данные. Пропусков и нулевых значений не обнаружено. Из интересного - доля токсичных комментариев от общего числа составила 10%.

Далее: 
  - Удалён ненужный стобец, дублирующий индексы;
  - Произведено преобразование текстовых данных (убраны лишние слова, сделана лемматизация);
  - Произведено разбиение на тренировочную, валидационную и тестовую выборки, выровнено количество "нулевых" таргетов в тренировочной выборке.
  
Данные полностью готовы для обучения моделей.

## Обучение

Для сравнения и выбора лучшейц обучим 4 модели: LogisticRegression, DecisionTreeClassifier, RandomForestClassifier, CatBoostClassifier.

Ради удобства и компактности напишем по одной ячейке для каждой модели. Сделаем предсказания на валидационных выборках, выберем лучшую модель по ним.

In [11]:
# Правка типа:
features_train = features_train.text

In [12]:
# LogisticRegression:

pipeline = Pipeline([("vect", TfidfVectorizer(stop_words='english', sublinear_tf=True)), 
                     ("lr", LogisticRegression())])
    
parameters = {'lr__solver': ('liblinear', 'saga', 'lbfgs'),
              'lr__C': (1, 5, 10),
              'lr__random_state': ([12345]),
              'lr__max_iter': ([200])}

# Обучение модели:

grid_search_lr = GridSearchCV(pipeline, parameters, scoring='f1', cv=3, n_jobs=-1)

grid_search_lr.fit(features_train, target_train)

mean_test_score = grid_search_lr.cv_results_['mean_test_score']
lr_train_f1 = max(mean_test_score)

print('F1 логистической регрессии:', round(lr_train_f1,2))
print('при параметрах', grid_search_lr.best_params_)
print()

# Валидационная выборка:
predictions_valid = grid_search_lr.predict(features_valid.text)
lr_valid_f1 = f1_score(target_valid, predictions_valid)
print('F1 логистической регрессии на валидации:', round(lr_valid_f1,2))

F1 логистической регрессии: 0.77
при параметрах {'lr__C': 10, 'lr__max_iter': 200, 'lr__random_state': 12345, 'lr__solver': 'lbfgs'}

F1 логистической регрессии на валидации: 0.78


In [13]:
# DecisionTreeClassifier:

pipeline = Pipeline([("vect", TfidfVectorizer(stop_words='english')), 
                     ("dtc", DecisionTreeClassifier())])
    
parameters = {'dtc__max_depth': ([x for x in range(1, 25)]),
              'dtc__random_state': ([12345])}

# Обучение модели:

grid_search_dt = GridSearchCV(pipeline, parameters, scoring='f1', cv=3, n_jobs=-1)

grid_search_dt.fit(features_train_resample, target_train_resample)

mean_test_score = grid_search_dt.cv_results_['mean_test_score']
dtc_train_f1 = max(mean_test_score)

print('F1 дерева решений:', round(dtc_train_f1,2))
print('при параметрах', grid_search_dt.best_params_)
print()

# Валидационная выборка:
predictions_valid = grid_search_dt.predict(features_valid.text)
dtc_valid_f1 = f1_score(target_valid, predictions_valid)
print('F1 дерева решений на валидации:', round(dtc_valid_f1,2))

F1 дерева решений: 0.72
при параметрах {'dtc__max_depth': 24, 'dtc__random_state': 12345}

F1 дерева решений на валидации: 0.63


In [14]:
# CatBoostClassifier:

pipeline = Pipeline([("vect", TfidfVectorizer(stop_words='english')), 
                     ("cbc", CatBoostClassifier())])
    
parameters = {'cbc__verbose': ([False]),
              'cbc__iterations': ([200]),
              'cbc__class_weights':([(1, 1), (1, 5)])}

# Обучение модели:

grid_search_cb = GridSearchCV(pipeline, parameters, scoring='f1', cv=3, n_jobs=-1)

grid_search_cb.fit(features_train_resample, target_train_resample)

mean_test_score = grid_search_cb.cv_results_['mean_test_score']
cbc_train_f1 = max(mean_test_score)

print('F1 CatBoostClassifier:', round(cbc_train_f1,2))
print('при параметрах', grid_search_cb.best_params_)
print()

# Валидационная выборка:
predictions_valid = grid_search_cb.predict(features_valid.text)
cbc_valid_f1 = f1_score(target_valid, predictions_valid)
print('F1 CatBoostClassifier на валидации:', round(cbc_valid_f1,2))

F1 CatBoostClassifier: 0.88
при параметрах {'cbc__class_weights': (1, 1), 'cbc__iterations': 200, 'cbc__verbose': False}

F1 CatBoostClassifier на валидации: 0.73


In [15]:
# RandomForestClassifier:

pipeline = Pipeline([("vect", TfidfVectorizer(stop_words='english')), 
                     ("rfc", RandomForestClassifier())])
    
parameters = {'rfc__n_estimators': ([x for x in range(10, 30)]),
              'rfc__random_state': ([12345]),
              'rfc__max_depth': ([x for x in range(1, 10)]),
              'rfc__criterion': (['entropy']),
              'rfc__class_weight': (['balanced'])}

# Обучение модели:

grid_search_rf = GridSearchCV(pipeline, parameters, scoring='f1', cv=3, n_jobs=-1)

grid_search_rf.fit(features_train_resample, target_train_resample)

mean_test_score = grid_search_rf.cv_results_['mean_test_score']
rfc_train_f1 = max(mean_test_score)

print('F1 случайного леса:', round(rfc_train_f1,2))
print('при параметрах', grid_search_rf.best_params_)
print()

# Валидационная выборка:
predictions_valid = grid_search_rf.predict(features_valid.text)
rfc_valid_f1 = f1_score(target_valid, predictions_valid)
print('F1 случайного леса на валидации:', round(rfc_valid_f1,2))

F1 случайного леса: 0.79
при параметрах {'rfc__class_weight': 'balanced', 'rfc__criterion': 'entropy', 'rfc__max_depth': 9, 'rfc__n_estimators': 29, 'rfc__random_state': 12345}

F1 случайного леса на валидации: 0.32


Для лучшего понимания ситуации сведём полученные результаты в таблицу.

In [16]:
# Создание сводки:
index = ['LogisticRegression',
         'DecisionTreeClassifier',
         'CatBoostClassifier',
         'RandomForestClassifier']

data = {'F1 на кросс-валидационной выборке': [lr_train_f1,
                                    dtc_train_f1,
                                    cbc_train_f1,
                                    rfc_train_f1],
        
        'F1 на валидационной выборке': [lr_valid_f1,
                                        dtc_valid_f1,
                                        cbc_valid_f1,
                                        rfc_valid_f1]}

f1_data = pd.DataFrame(data=data, index=index)

display(f1_data.sort_values(by='F1 на валидационной выборке', ascending=False))

,F1 на обучающей выборке,F1 на валидационной выборке
LogisticRegression,0.771768,0.782548
CatBoostClassifier,0.876559,0.730687
DecisionTreeClassifier,0.720390,0.629715
RandomForestClassifier,0.788554,0.323490


### Вывод:
В ходе работы были обучены 4 модели (LogisticRegression, CatBoostClassifier	, DecisionTreeClassifier, RandomForestClassifier), каждая сделала предсказания по валидационной выборке, результаты размещены в итоговую таблицу. Как нередко бывает, самое лучшее решение оказалось самым простым - логистическая регрессия справилась с задачей лучше всех.

## Выводы

Посмотрим предсказания лучшей модели на тестовой выборке и проверим её результат по метрике F1.

In [17]:
# Предсказания лучшей модели:
predictions_test = grid_search_lr.predict(features_test.text)
lr_test_f1 = f1_score(target_test, predictions_test)
print('F1 логистической регрессии на тесте:', round(lr_test_f1,2))

F1 логистической регрессии на тесте: 0.79


Модель справилась с поставленной задачей.

### Итоговые выводы:
На разбор были предоставлены данные из "Викишопа" с комментариями пользователей. Задача заключалась в создании модели МО для контроля токсичности данных комментариев.

Были загружены и осмотрены предоставленные данные. Пропусков и нулевых значений не обнаружено. Из интересного - доля токсичных комментариев от общего числа составила 10%.

Далее: 
  - Удалён ненужный стобец, дублирующий индексы;
  - Произведено преобразование текстовых данных (убраны лишние слова, сделана лемматизация);
  - Произведено разбиение на тренировочную, валидационную и тестовую выборки, выровнено количество "нулевых" таргетов в тренировочной выборке.
  
В ходе работы были обучены 4 модели (LogisticRegression, CatBoostClassifier , DecisionTreeClassifier, RandomForestClassifier), каждая сделала предсказания по валидационной выборке, результаты размещены в итоговую таблицу. Как нередко бывает, самое лучшее решение оказалось самым простым - логистическая регрессия справилась с задачей лучше всех.

Результаты модели по тестовой выборке также оказались удовлетворительными (F1 = 0,79).

Поставленная задача была выполнена в полном объёме, дополнительных исследований не требуется.